<a href="https://colab.research.google.com/github/Prithiksha-V/hackton/blob/main/phase3ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install pyngrok
!pip install flask pandas scikit-learn

# Upload your CSV
from google.colab import files
uploaded = files.upload()

# Load CSV
import io
import pandas as pd
filename = list(uploaded.keys())[0]
data = pd.read_csv(io.BytesIO(uploaded[filename]))

# Train model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

X = data[['sqft', 'bedrooms', 'bathrooms']]
y = data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Set up Flask app
from flask import Flask, request, render_template_string

app = Flask(__name__)

html = '''
<!DOCTYPE html>
<html>
<head>
    <title>House Price Predictor</title>
    <style>
        body { font-family: Arial; text-align: center; }
        form { margin: auto; width: 300px; }
        input, button { width: 100%; padding: 10px; margin: 5px; }
        button { background: green; color: white; border: none; }
    </style>
</head>
<body>
    <h2>House Price Predictor</h2>
    <form method="post">
        <input name="sqft" type="number" step="any" placeholder="Square Feet" required>
        <input name="bedrooms" type="number" placeholder="Bedrooms" required>
        <input name="bathrooms" type="number" placeholder="Bathrooms" required>
        <button type="submit">Predict Price</button>
    </form>
    {% if price %}
        <h3>Estimated Price: ₹{{ price }}</h3>
    {% endif %}
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    price = None
    if request.method == 'POST':
        sqft = float(request.form['sqft'])
        bedrooms = int(request.form['bedrooms'])
        bathrooms = int(request.form['bathrooms'])
        features = np.array([[sqft, bedrooms, bathrooms]])
        price = round(model.predict(features)[0], 2)
    return render_template_string(html, price=price)

# Run Flask with pyngrok
from pyngrok import ngrok

port = 5000
public_url = ngrok.connect(port)
print(f"App is live at: {public_url}")

app.run(port=port)
